# This notebook as been used to scrap all moodle files

In [2]:
# get credentials from venv 
import os 
moodle_url = os.getenv('moodle_url')
username = os.getenv('username')
password = os.getenv('password')

The part below is used to setup selenium web driver to scrap moodle files

In [8]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ✅ Setup the download directory
download_folder = os.path.abspath("moodle_downloads")
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# ✅ Setup the text file for saving links progressively
links_file_path = os.path.join(download_folder, "moodle_links.txt")

# ✅ Chrome Options: Disable PDF Viewer and Force Download
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": download_folder,  
    "download.prompt_for_download": False,  
    "download.directory_upgrade": True,  
    "plugins.always_open_pdf_externally": True,  
    "safebrowsing.enabled": True  
})


In [29]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)  

# ✅ Step 1: Login Function
def login():
    driver.get(moodle_url)

    try:
        oauth_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'auth/oauth2')]"))
        )
        oauth_button.click() #As we said here's the part when you get redirected to MyEFrei portal and you have to fill it with your credentials
        print("Redirecting to the portal...")
    except:
        print("OAuth2 button not detected. Attempting direct login...")

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username) #here we fill the username field
    driver.find_element(By.ID, "password").send_keys(password) #here we fill the password field
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    print("Successfully logged in!")
    time.sleep(5)

# ✅ Fetch all categories
def get_all_categories():
    return driver.find_elements(By.XPATH, "//h4[@class='categoryname aabtn']/a") #First when we arrived to home page when fetch all possible categories.

# ✅ Fetch all courses within a category
def get_all_courses():
    return driver.find_elements(By.XPATH, "//h3[@class='coursename']/a") #As we said it's a complex tree, inside the categories we have courses and inside the courses we have the files.

# ✅ Fetch all downloadable files within a course and write to file progressively
def get_all_downloads():# We don't directly download the data in order to be more efficient we juste keep the download link
    file_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'mod/resource/view.php')]")
    urls = [link.get_attribute("href") for link in file_links]

    # ✅ Write each link to the file progressively
    with open(links_file_path, "a") as file:
        for url in urls:
            file.write(f"{url}\n")
    return urls

# ✅ Main scraping function with better navigation handling
def scrape_all_files(): #Finnaly we can start the scraping
    login()
    categories = get_all_categories()
    len_categories = len(categories)

    for i in range(len_categories):
        categories = get_all_categories()
        category_link = categories[i].get_attribute('href')
        print("link",category_link)
        driver.get(category_link)
        time.sleep(2)

        courses = get_all_courses()
        for course in courses:
            course_link = course.get_attribute('href')
            driver.get(course_link)
            time.sleep(2)

            # ✅ Fetch files and write links progressively
            get_all_downloads()

            # Navigate back safely
            driver.back()
            time.sleep(2)

        # Navigate back to categories
        driver.get(moodle_url)
        time.sleep(2)


def partial_scrape_all_files():
    login()
    categories = valid_categories_links
    len_categories = len(categories)
    for i in range(len_categories):
        categories = get_all_categories()
        category_link = categories[i].get_attribute('href')
        print("link",category_link)
        driver.get(category_link)
        time.sleep(2)

        courses = get_all_courses()
        for course in courses:
            course_link = course.get_attribute('href')
            driver.get(course_link)
            time.sleep(2)

            # ✅ Fetch files and write links progressively
            get_all_downloads()

            # Navigate back safely
            driver.back()
            time.sleep(2)

        # Navigate back to categories
        driver.get(moodle_url)
        time.sleep(2)

def record_categories():
    login()
    all_categories = get_all_categories()
    with open('categories.txt', 'w') as file:
        for category in all_categories:
            file.write(f"{category.get_attribute('href')}\n")
    print("Categories have been saved to categories.txt")



In [ ]:
# ✅ Initialize the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)  
try:
    record_categories()

except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit()

In [ ]:
valid_categories_links = []
# read categories .txt and take first 6 lines 
with open('categories.txt') as f:
    valid_categories_links = f.readlines()
valid_categories_links = [x.strip() for x in valid_categories_links]
valid_categories_links = valid_categories_links[:6]

print(valid_categories_links)

# Then this part is when we have all the url to download and we just want to donwload them

In [33]:
moodle_links = "moodle_links.txt"
download_folder = "moodle_downloads"  
def download_folder():
    login()
    with open(moodle_links) as f:
        download_links = f.readlines()
        download_links = [x.strip() for x in download_links]
        for link in download_links:
            driver.get(link)
            time.sleep(2)        


In [ ]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
try:
    # partial_scrape_all_files()
    download_folder()
except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit()

Redirecting to the portal...
Successfully logged in!
